![image](https://cdn.pixabay.com/photo/2020/01/08/14/50/music-4750275_960_720.jpg)

# Using SQL to Respond Business Questions: Analyzing the Chinook Database
*Author: David Rodrigues. davidrodriguessp@hotmail.com. in/davidrodrigues. January, 2021.*



To build this project we used a database called Chinook. This is a fictitious database build to represent a digital music shop. It countains information like artists, songs and albums sold at the digital store. The database includes 11 tables with different data about the business.

The Chinook database was downloaded from [Dataquest](https://www.dataquest.io/). The database file called `chinook-unmodified.db` is available at my [github repository](https://github.com/davidrodriguessp/sql_respond_business_questions).

The diagram below, elaborated by Dataquest, depicts how the tables and columns are organized.

## Chinook Database Diagram

![Image](chinook_schema.png)

## Reading and Exploring the Database

The database should be saved in the same directory the Jupyter Notebook is located. The code below was run to connect this notebook to the database file.

In [1]:
%%capture
%load_ext sql
%sql sqlite:///chinook-unmodified.db

Next, we run a query to get a list of all tables and views in the database. To run SQL queries in a Jupyter Notebook, we have to add `%%sql` on its own line to the start of each query.

In [2]:
%%sql
SELECT
    name,
    type
  FROM sqlite_master
  WHERE type IN ("table","view");

 * sqlite:///chinook-unmodified.db
Done.


name,type
album,table
artist,table
customer,table
employee,table
genre,table
invoice,table
invoice_line,table
media_type,table
playlist,table
playlist_track,table


We see the same 11 tables we had depicted in the diagram previously.

We will start by exploring the columns of two tables: `customer` and `invoice`.

In [3]:
%%sql
SELECT * 
  FROM customer 
 LIMIT 2;

 * sqlite:///chinook-unmodified.db
Done.


customer_id,first_name,last_name,company,address,city,state,country,postal_code,phone,fax,email,support_rep_id
1,Luís,Gonçalves,Embraer - Empresa Brasileira de Aeronáutica S.A.,"Av. Brigadeiro Faria Lima, 2170",São José dos Campos,SP,Brazil,12227-000,+55 (12) 3923-5555,+55 (12) 3923-5566,luisg@embraer.com.br,3
2,Leonie,Köhler,None,Theodor-Heuss-Straße 34,Stuttgart,None,Germany,70174,+49 0711 2842222,None,leonekohler@surfeu.de,5


In [4]:
%%sql
SELECT * 
  FROM invoice 
 LIMIT 2;

 * sqlite:///chinook-unmodified.db
Done.


invoice_id,customer_id,invoice_date,billing_address,billing_city,billing_state,billing_country,billing_postal_code,total
1,18,2017-01-03 00:00:00,627 Broadway,New York,NY,USA,10012-2612,15.84
2,30,2017-01-03 00:00:00,230 Elgin Street,Ottawa,ON,Canada,K2P 1L7,9.9


The two tables have a common key: `customer_id`. We will use this column to connect this two tables.

## Which Genres Sell the Most in the USA?
We want to understand, from the Chinook Record Company perspective, which type of artist would have more potential for sales in the USA. Having this in mind, we will explore how different genres perform in terms of sales.

If we refer back to the database Diagram, we see that there are three tables we will need to use to perform this analysis: `genre`, `track`and `customer`. We will need to connect the three tables to be able to identify different tracks, genres and countries.

In [5]:
%%sql
WITH
all_usa_tracks AS
    (SELECT 
         g.name genre,
         t.track_id track_id
         FROM invoice_line ii
         INNER JOIN invoice i ON i.invoice_id = ii.invoice_id
         INNER JOIN customer c ON c.customer_id = i.customer_id
         INNER JOIN track t ON ii.track_id = t.track_id
         INNER JOIN genre g ON g.genre_id = t.genre_id
         WHERE c.country = 'USA')
SELECT
    genre,
    COUNT(track_id) track_number,
    ROUND(CAST(COUNT(track_id) AS Float) / 
        (SELECT 
         COUNT(track_id) FROM all_usa_tracks), 3) percentage
    FROM all_usa_tracks
    GROUP BY genre
    ORDER BY percentage DESC
    LIMIT 10;

 * sqlite:///chinook-unmodified.db
Done.


genre,track_number,percentage
Rock,561,0.534
Alternative & Punk,130,0.124
Metal,124,0.118
R&B/Soul,53,0.05
Blues,36,0.034
Alternative,35,0.033
Latin,22,0.021
Pop,22,0.021
Hip Hop/Rap,20,0.019
Jazz,14,0.013


We see that more than half of the tracks sold in the platform are from Rock bands. This means that new Rock musicians might have potential for sales in the USA, of course, depending on the quality of their work.

## Sales Support Agents Performance
Next, we will analyze how each of the Sales Support Agents perform. We will need to connect the `consumer`, `invoice` and `employee` tables to carry out this analysis.

In [6]:
%%sql
SELECT * FROM employee
WHERE title = 'Sales Support Agent';

 * sqlite:///chinook-unmodified.db
Done.


employee_id,last_name,first_name,title,reports_to,birthdate,hire_date,address,city,state,country,postal_code,phone,fax,email
3,Peacock,Jane,Sales Support Agent,2,1973-08-29 00:00:00,2017-04-01 00:00:00,1111 6 Ave SW,Calgary,AB,Canada,T2P 5M5,+1 (403) 262-3443,+1 (403) 262-6712,jane@chinookcorp.com
4,Park,Margaret,Sales Support Agent,2,1947-09-19 00:00:00,2017-05-03 00:00:00,683 10 Street SW,Calgary,AB,Canada,T2P 5G3,+1 (403) 263-4423,+1 (403) 263-4289,margaret@chinookcorp.com
5,Johnson,Steve,Sales Support Agent,2,1965-03-03 00:00:00,2017-10-17 00:00:00,7727B 41 Ave,Calgary,AB,Canada,T3B 1Y7,1 (780) 836-9987,1 (780) 836-9543,steve@chinookcorp.com


As we can see above, there are only three Sales Support Agents. The three were hired in 2017 and are based in the city of Calgary, in Canada . 

The only difference seems to be their age. Jane is the yougest, 8 years younger than Steve and 26 years younger than Margareth. Their profile is similar in the other characteristics.

Let's make a query to understand until when we have data related to sales. Then, we will decide how to analyze the data.

In [7]:
%%sql
SELECT * 
  FROM invoice
  ORDER BY invoice_date DESC
  LIMIT 5;

 * sqlite:///chinook-unmodified.db
Done.


invoice_id,customer_id,invoice_date,billing_address,billing_city,billing_state,billing_country,billing_postal_code,total
614,53,2020-12-30 00:00:00,113 Lupus St,London,None,United Kingdom,SW1V 3EN,12.87
613,20,2020-12-29 00:00:00,541 Del Medio Avenue,Mountain View,CA,USA,94040-111,8.91
611,52,2020-12-27 00:00:00,202 Hoxton Street,London,None,United Kingdom,N1 5LH,1.98
612,33,2020-12-27 00:00:00,5112 48 Street,Yellowknife,NT,Canada,X1A 1N6,11.879999999999999
610,55,2020-12-21 00:00:00,421 Bourke Street,Sidney,NSW,Australia,2010,6.93


We see that sales are updated until December 2020. Let's analyze the sales performed in the year 2020.

In [8]:
%%sql
WITH joined_table AS
    (SELECT 
            e.first_name || ' ' || e.last_name agent,
            ROUND(SUM(i.total), 2) total_sales
        FROM employee e
        INNER JOIN customer c ON e.employee_id = c.support_rep_id
        INNER JOIN invoice i ON i.customer_id = c.customer_id
        WHERE title = 'Sales Support Agent' AND i.invoice_date LIKE '2020-%'
        GROUP BY 1
        ORDER BY 2 DESC)

SELECT * 
  FROM joined_table 
  LIMIT 10;

 * sqlite:///chinook-unmodified.db
Done.


agent,total_sales
Jane Peacock,401.94
Steve Johnson,371.25
Margaret Park,365.31


We see that Jane has better results than the other two Sales Reps in 2020. Her sales are around 10% higher if compared to Margareth's, for example.

In future queries we will use the `GROUP BY` and `ORDER BY` clauses. Below we show a query that exemplifies how they are used. In the query below we measure which are the most common first names among the Chinook shop customers. 

In [9]:
%%sql
SELECT 
        first_name, 
        COUNT(customer_id) 
    FROM customer 
    GROUP BY 1 
    ORDER BY 2 DESC 
    LIMIT 5;

 * sqlite:///chinook-unmodified.db
Done.


first_name,COUNT(customer_id)
Frank,2
Mark,2
Aaron,1
Alexandre,1
Astrid,1


## Results per country
In the next query, we will verify how different countries perform in terms of customers and sales.

In [10]:
%%sql
WITH customer_invoice AS
    (SELECT
        c.customer_id customer_id,
        c.country country,
        i.invoice_id invoice_id,
        i.total sales
      FROM invoice i
     INNER JOIN customer c ON c.customer_id = i.customer_id),

other_countries AS
    (SELECT
      country,
      CASE
         WHEN COUNT(DISTINCT customer_id) = 1 then 'Other'
         ELSE country
         END AS country_new,
      COUNT(DISTINCT customer_id) count
      FROM customer_invoice
      GROUP BY country
      ORDER BY count DESC),

countries_with_others AS
    (SELECT 
         other_countries.count num_customers, 
         customer_invoice.country country,
         other_countries.country_new country_with_others,
         customer_invoice.customer_id customer_id,
         customer_invoice.invoice_id invoice_id,
         customer_invoice.sales sales
    FROM customer_invoice 
    INNER JOIN other_countries ON customer_invoice.country = other_countries.country)

SELECT
    country_with_others countries,
    COUNT(DISTINCT customer_id) total_customers,
    ROUND(SUM(sales), 2) total_sales,
    ROUND(CAST(SUM(sales) AS Float) / COUNT(DISTINCT invoice_id), 2) average_order,
    ROUND(CAST(SUM(sales) AS Float) / COUNT(DISTINCT customer_id), 2) average_sales_customer,
        CASE
            WHEN country_with_others = 'Other' THEN 0
            ELSE 1
            END AS sort
     FROM countries_with_others
     GROUP BY country_with_others
     ORDER BY sort DESC, total_sales DESC;

 * sqlite:///chinook-unmodified.db
Done.


countries,total_customers,total_sales,average_order,average_sales_customer,sort
USA,13,1040.49,7.94,80.04,1
Canada,8,535.59,7.05,66.95,1
Brazil,5,427.68,7.01,85.54,1
France,5,389.07,7.78,77.81,1
Germany,4,334.62,8.16,83.66,1
Czech Republic,2,273.24,9.11,136.62,1
United Kingdom,3,245.52,8.77,81.84,1
Portugal,2,185.13,6.38,92.57,1
India,2,183.15,8.72,91.57,1
Other,15,1094.94,7.45,73.0,0


The analysis reveals that the USA is the country with most customers. However, there seems to be an enormous untapped opportunity in Czech Republic. 

There are still only 2 customers in this country. However, the average sales per customer is much higher than in the USA and more than two times higher than in Canada.

## Measuring if Customers Buy More Often Full Albums or Individual Tracks
In our last query, we need to verify the percentage of customers that bought complete albums and the ones that bought individual tracs.

When we analyze the database, we see that this information is not directly available. At the same time, company policies do not allow customers to buy in the same purchase a full album, and add individual tracks.

Therefore, if we analyze different invoices, there are only two options:
- Invoices that include a full album with all its tracks
- Invoices that include only individual tracks, and no full album

To respond this question we will build a query that:
1. Joins together the tables invoice, track and album in a new table called `complete``
2. Create a table called `track_actual` with the invoices and the list of tracks purchased in each one
3. Use the first track of the invoice to identify the album that track belongs to. Then, we created the tables `invoices_first_track` and then the table `invoices_album`. This last one lists for each invoice what would be the list of tracks *if* the invoice included really a full album.
4. Then, we created the table `album_actual` joining the list of the actual tracks per invoice together with the list of tracks that the invoice would have *if* it was an album track.
5. Finally, we created the table `equal_album` checking for each invoice if it was a full album purchase (Yes) or a purchase of individual tracks (No).

In [11]:
%%sql
WITH 
complete AS
    (
     SELECT 
        i.invoice_id invoice,
        t.track_id track,
        t.album_id album
    FROM track t
    INNER JOIN invoice_line il ON il.track_id = t.track_id
    INNER JOIN invoice i ON i.invoice_id = il.invoice_id
    ),
invoice_list AS
    (
    SELECT
        DISTINCT invoice invoice_list
    FROM complete
    ),
track_actual AS
    (
    SELECT
        invoice invoice_actual,
        track track_actual
      FROM complete
    ),

invoices_first_track AS
    (
    SELECT
        invoice,
        MIN(track) first_track
      FROM complete
      GROUP BY invoice
    ),

invoices_album AS
    (
    SELECT
        ift.invoice invoice,
        t.album_id album
      FROM invoices_first_track ift
      INNER JOIN track t ON t.track_id = ift.first_track
    ),

track_album AS
    (
    SELECT 
        ia.invoice invoice_album,
        t.track_id track_album
      FROM invoices_album ia
      INNER JOIN track t ON t.album_id = ia.album
    ),

album_actual AS
    (
    SELECT 
        tal.invoice_album,
        tal.track_album,  
        tac.invoice_actual,
        tac.track_actual 
      FROM track_album tal
      INNER JOIN track_actual tac ON tac.invoice_actual = tal.invoice_album
      ORDER BY invoice_album ASC, invoice_actual ASC
    ),
equal_album AS
    (SELECT
    tac.invoice_actual invoice,
        CASE
            WHEN
            (
            SELECT track_album FROM track_album WHERE invoice_album = tac.invoice_actual
            EXCEPT
            SELECT track_actual FROM track_actual WHERE invoice_actual = tac.invoice_actual
            ) IS NULL
            AND
            (
            SELECT track_actual FROM track_actual WHERE invoice_actual = tac.invoice_actual
            EXCEPT
            SELECT track_album FROM track_album WHERE invoice_album = tac.invoice_actual
            ) IS NULL
            THEN 'Yes'
            ELSE 'No'
            END AS 'equal_album'
        FROM track_actual tac
        INNER JOIN track_album tal ON tal.invoice_album = tac.invoice_actual
        GROUP BY tac.invoice_actual)

SELECT
    equal_album, 
    COUNT(equal_album) Count,
    ROUND(CAST(COUNT(equal_album) AS Float) / 
          (SELECT COUNT(DISTINCT invoice) FROM complete), 2) Percentage
  FROM equal_album 
  GROUP BY equal_album;

 * sqlite:///chinook-unmodified.db
Done.


equal_album,Count,Percentage
No,500,0.81
Yes,114,0.19


As we can see in the table below, there is a considerable portion of purchases that are full albums (19%). Therefore, it is important that the business considers this for their future decidions on how to define their purchases from record companies.

For example, the idea of not buying complete albums and focus only in buying the most popular tracks might not be profitable since it would most probably impact nearly 20% of the volume in terms of tracks purchased.